The Movies Dataset

Эти файлы содержат метаданные для всех 45 000 фильмов, перечисленных в полном наборе данных MovieLens. Набор данных состоит из фильмов, выпущенных в июле 2017 года или ранее. Данные включают актерский состав, съемочную группу, ключевые слова сюжета, бюджет, доходы, постеры, даты выхода, языки, производственные компании, страны, количество голосов на TMDB и средние значения голосов.

movies_metadata.csv - Основной файл метаданных фильмов. Содержит информацию о 45 000 фильмах, представленных в полном наборе данных MovieLens. Характеристики включают постеры, декорации, бюджет, доходы, даты выхода, языки, страны и компании-производители.

Цель этого проекта - проанализировать метаданные фильмов, представленные в файле 'movies_metadata.csv':

1) Очистить и предварительно обработать данные, преобразовав столбец 'adult' в булевы значения, а столбец 'release_date' - в формат времени.
2) Изучить набор данных, проанализировав количество фильмов, их жанры и другую необходимую информацию.
3) Произвести статистический анализ, чтобы выявить тенденции и закономерности в данных, например распределение фильмов по жанрам, годам выпуска и продолжительности.
4) Визуализация данных для лучшего понимания тенденций и закономерностей, выявленных в ходе статистического анализа.
5) Ответить на конкретные вопросы о данных, например о самом популярном жанре, среднем времени проката фильмов и распределении фильмов по годам выпуска.
6) Предоставить рекомендации по дальнейшему анализу или пониманию того, что можно получить из полученных данных.
В целом, цель данного проекта - провести всесторонний анализ метаданных о фильмах и получить сведения, которые можно использовать для лучшего понимания киноиндустрии.

Import necessary libraries: The script imports several libraries, including pandas, numpy,aborn, matlib, plotly, xgboost, andlearn. 2 Load the data: The script reads in a CSV file containing movies' metadata and assigns it to a pandas DataFrame called movies. It also performs some data cleaning tasks, such as converting the adult column to a boolean type and the release_date column to a datetime type.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import ast
# Import data
movies = pd.read_csv(r'C:\IDE\movies_metadata.csv')
movies['adult'] = movies['adult'].map({'True': True, 'False': False})
movies['release_date'] = pd.to_datetime(movies['release_date'], errors='coerce')

Visualize missing values: The script creates a heatmap to visualize the missing values in the dataset.


In [ ]:
# Visualize missing values
plt.figure(figsize=(10, 6))
sns.heatmap(movies.isnull(), cbar=False, cmap='viridis')
plt.title('Missing Values Visualization')
plt.show()

Clean the data: The script defines a function called clean_data that drops any rows with missing values in the release_date, budget, or revenue columns. It then applies this function to the movies DataFrame.


In [ ]:
# Clean data
def clean_data(df):
    return df.dropna(subset=['release_date', 'budget', 'revenue'], axis=0).reset_index(drop=True)

movies = clean_data(movies)

Convert the success_or_flop column to numeric values: The script maps the values in the success_or_flop column to numeric values (0 for failures and 1 for hits).


In [ ]:
# Convert success_or_flop column to numeric values
movies['popularity'] = movies['popularity'].map({'failure': 0, 'hit': 1})

Convert relevant columns to numeric or datetime types: The script defines a function called convert_types that converts the id, budget, popularity, and release_date columns to numeric or datetime types. It then applies this function to the movies DataFrame.


In [ ]:
# Convert relevant columns to numeric or datetime types
def convert_types(df, columns):
    for col in columns:
        if col == 'id':
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='integer')
        elif col in ['budget', 'popularity']:
            df[col] = pd.to_numeric(df[col], errors='coerce', downcast='float')
        else:
            df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

convert_types(movies, ['id', 'budget', 'popularity', 'release_date'])

Create new columns for the year and month of release date: The script creates two new columns in the movies DataFrame: release_year and release_month.


In [ ]:
movies['release_year'] = movies['release_date'].dt.year
movies['release_month'] = movies['release_date'].dt.month

Visualize the correlation matrix: The script creates a heatmap to visualize the correlation matrix of the budget, popularity, release_year, and release_month columns.


In [ ]:
correlation_matrix = movies[['budget', 'popularity', 'release_year', 'release_month']].corr()
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Matrix')
plt.show()

Visualize the budget and revenue distribution: The script creates a histogram to visualize the distribution of the budget and revenue columns.


In [ ]:
plt.figure(figsize=(12, 6))
sns.histplot(movies['budget'], bins=30, kde=True, color='blue', label='Budget')
sns.histplot(movies['revenue'], bins=30, kde=True, color='green', label='Revenue')
plt.title('Budget and Revenue Distribution')
plt.xlabel('Amount (in million)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

Visualize the release year distribution: The script creates a count plot to visualize the distribution of the release_year column.


In [ ]:
# Visualize release year distribution
plt.figure(figsize=(12, 6))
sns.countplot(x='release_year', data=movies, palette='viridis')
plt.title('Release Year Distribution')
plt.xlabel('Release Year')
plt.ylabel('Count')
plt.show()

Convert the genres column to list type: The script converts the genres column to a list type and explodes the list into separate rows.


In [ ]:
movies['genres'] = movies['genres'].apply(ast.literal_eval)
movies['genres'] = movies['genres'].apply(lambda x: [item for sublist in x for item in sublist])

Visualize the top genres: The script creates a bar plot to visualize the top 10 genres in the dataset.


In [ ]:
genres_list = movies['genres'].explode().value_counts().head(10)

plt.figure(figsize=(12, 6))
sns.barplot(x=genres_list.values, y=genres_list.index, palette='viridis')
plt.title('Top 10 Genres')
plt.xlabel('Count')
plt.ylabel('Genre')
plt.show()

Create a new column for success or flop: The script creates a new column called success_or_flop based on whether the revenue is greater than the budget.


In [ ]:
# Create success_or_flop column based on revenue and budget
movies['success_or_flop'] = np.where(movies['revenue'] > movies['budget'], 1, 0)

Train an XGBoost model: The script splits the data into training and testing sets and trains an XGBoost classifier on the training set.


In [ ]:
# XGBoost model training
X = movies[['budget', 'popularity', 'release_year', 'release_month']]
y = movies['success_or_flop']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)

Evaluate the model: The script evaluates the model's performance on the testing set by calculating the accuracy score, classification report, and confusion matrix.


In [ ]:
# Model evaluation
y_pred = model.predict(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='viridis')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

В целом этот сценарий выполняет различные задачи по очистке, исследованию и анализу данных в наборе метаданных фильмов и обучает модель машинного обучения предсказывать успех или провал фильма на основе его бюджета, популярности, года выпуска и месяца выхода.